### 导入需要的包

In [1]:
!apt update
!apt install chromium-chromedriver

# 检查环境中是否有selenium/webdriver_manager，若无，则直接安装
# 安装依赖
!pip install selenium
!pip install webdriver_manager

# https://developers.google.com/drive/api/v3/quickstart/python
!pip install pygsheets
!pip install -U google-api-python-client google-auth-httplib2 google-auth-oauthlib


No Java runtime present, requesting install.
No Java runtime present, requesting install.
Requirement already up-to-date: google-api-python-client in /opt/anaconda3/lib/python3.8/site-packages (1.12.8)
Requirement already up-to-date: google-auth-httplib2 in /opt/anaconda3/lib/python3.8/site-packages (0.0.4)
Requirement already up-to-date: google-auth-oauthlib in /opt/anaconda3/lib/python3.8/site-packages (0.4.2)


In [2]:
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
import time
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains

In [3]:
# 这一块代码，大家可以不需要了解意思，只要 copy 过来就好了
from __future__ import print_function
from pathlib import Path
import os
import io
import pickle
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from apiclient import errors
from googleapiclient.http import MediaIoBaseDownload

import pygsheets
import pandas as pd

# If modifying these scopes, delete the file token.pickle.
SCOPES = [
    'https://www.googleapis.com/auth/spreadsheets',
    'https://www.googleapis.com/auth/drive'
]

# add profile just for support different google accounts
def get_creds(profile=""):
    """Shows basic usage of the Drive v3 API.
    Prints the names and ids of the first 10 files the user has access to.
    """
    creds = None
    # The file token.pickle stores the user's access and refresh tokens, and is
    # created automatically when the authorization flow completes for the first
    # time.
    home = str(Path.home())
    google_helper = os.path.join(home,".google_helper")
    print(google_helper)
    Path(google_helper).mkdir(parents=True, exist_ok=True)
    
    if len(profile)>0:
        token_path = os.path.join(home,".google_helper","{}_token.pickle".format(profile))
        credential_path = os.path.join(home,".google_helper","{}_credentials.json".format(profile))
    else:
        token_path = os.path.join(home,".google_helper","token.pickle")
        credential_path = os.path.join(home,".google_helper","credentials.json")

    if not os.path.exists(credential_path):
        print("Google 授权文件:{}, 不存在!".format(credential_path))
        print("请把 credentials.json 放在 {} 这个目录下 :".format(google_helper))
        return None
        
    if os.path.exists(token_path):
        with open(token_path, 'rb') as token:
            creds = pickle.load(token)
    
    # If there are no (valid) credentials available, let the user log in.
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                credential_path, SCOPES)
            creds = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open(token_path, 'wb') as token:
            pickle.dump(creds, token)
    return creds
# 得到 google drive/sheet 的权限
creds = get_creds()
if creds is None:
    print("请配好权限！")

/Users/anker/.google_helper


In [4]:
chrome_options = webdriver.ChromeOptions()
###chrome_options.add_argument('--user-data-dir=chrome-data') # 使用chrome缓存的用户信息

# 其它使用ChromeDriverManager(要求系统中有chrome浏览器)
driver = webdriver.Chrome(ChromeDriverManager().install(), options=chrome_options)

# 最大化窗口
driver.maximize_window()
# 全屏显示
#driver.fullscreen_window()

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - There is no [mac64] chromedriver for browser 87.0.4280 in cache
[WDM] - Get LATEST driver version for 87.0.4280


[WDM] - Trying to download new driver from http://chromedriver.storage.googleapis.com/87.0.4280.88/chromedriver_mac64.zip
[WDM] - Driver has been saved in cache [/Users/anker/.wdm/drivers/chromedriver/mac64/87.0.4280.88]


## 主程序

In [5]:
## 定义访问的链接
## 定位到美国站点，本来希望访问bestbuy.com，再选择美国站点，但是定位不到美国站点选择的button,只好直接使用了美国站点的网页链接。
HOME_URL = 'https://www.bestbuy.com/?intl=nosplash'
## 访问链接
driver.get(HOME_URL)

## 关掉弹出来的广告?
ad_path = '//*[@id="widgets-view-email-modal-mount"]/div/div/div[1]/div/div/div/div/button'
driver.find_element_by_xpath(ad_path).click()

In [6]:
# 搜索/
def search_brand(brand):
    ## 定位到搜索框  //*[@id="gh-search-input"]
    search_input_path = '//*[@id="gh-search-input"]'
    search_input = driver.find_element_by_xpath(search_input_path)
    time.sleep(3)
    ## 输入要搜索的关键词
    search_input.send_keys(brand)
    ## 点击搜索 //*[@id="header-block"]/div[2]/div[1]/div/div[2]/div/div[1]/div/div/form/button[2]
    search_click_path = '//*[@id="header-block"]/div[2]/div[1]/div/div[2]/div/div[1]/div/div/form/button[2]'
    driver.find_element_by_xpath(search_click_path).click()
    return


# 读取谷歌文件
def read_google(filePath,sheet_Name):
    gc = pygsheets.authorize(custom_credentials=creds)
    sh = gc.open_by_url(filePath)
    current_sheet = sh.worksheet_by_title(sheet_Name)
    return current_sheet


## 将抓取到的信息封装成一个字典
def get_sku_item_info(element_item):
    sku_title = element_item.find_element_by_xpath(".//*[@class='sku-header']").text
    sku = element_item.find_element_by_xpath(".//*[@class='sku-value']").text
    sale_price = element_item.find_element_by_xpath(".//*[@class='priceView-hero-price priceView-customer-price']/span[1]").text
    regular_price = 0
    try:
        regular_price = element_item.find_element_by_xpath(".//*[@class='pricing-price__regular-price']").text
    except:
        regular_price = sale_price
        
    ##  可以在这里继续加数据 抓取剩余的数据,比如图片
    return {"sku_title":sku_title,"sku":sku,"sale_price":sale_price,"regular_price":regular_price}

   

In [ ]:
## 没有翻页的


## 现在可以手动修改brand和work sheet名字，后续可以写一个for函数，去https://docs.google.com/spreadsheets/d/1XzBC6LqIQ0S_ZerI6tqgV17my9G5N92cQSZdWeZaGBw/edit#gid=1844979788读取表格 然后循环查询不同的brand,这样的话还需要清空原来的搜索框
brand = 'eufy Security'
search_brand(brand)
## 搜索后默认加等待时间，防止网页还没加载完成报错
time.sleep(10)

sku_list = []
result_element =  driver.find_elements_by_xpath('//*[@id="main-results"]/ol/li')
## 注意try except的用法
for element_item in result_element:
    try:
        sku_list.append(get_sku_item_info(element_item))
    except:
        print("Main Results Continued")

## 利用字典 拼接df数据 https://blog.csdn.net/u010199356/article/details/85697860 学会查资料
df = pd.DataFrame(sku_list, columns = ["sku_title", "sku", "sale_price","regular_price"])

# 读取Google Sheet并写入(现在是完全覆盖，以后想写成标注好日期，每天在后面接着前一天的数据写)
sheet = read_google('https://docs.google.com/spreadsheets/d/1XzBC6LqIQ0S_ZerI6tqgV17my9G5N92cQSZdWeZaGBw/edit#gid=0','eufy Security')
sheet.set_dataframe(df,(1,1))
df

In [ ]:
check_price = "$199.8"
check_price_reg = "Reg $1999.99"
print(check_price.replace("$",""))
print(check_price_reg.replace("$","").replace("Reg","").replace(" ",""))
float(check_price_reg.replace("$","").replace("Reg","").replace(" ",""))-float(check_price.replace("$",""))